In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-10-16 10:05 ajna         INFO     Configuração de log efetuada


2025-10-16 10:05:42,017 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id,
    c.placa,
    c.placaSemirreboque,
    c.numeroConteiner,
    c.vazioConteiner,
    c.codigoRecinto,
    c.cpfMotorista,
    c.nomeMotorista,
    c.dataHoraOcorrencia,
    c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner 
    AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia >= c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (
        (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
        OR e.codigoRecinto = '8931404'
    )
    AND e.codigoRecinto != c.codigoRecinto
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# A tabela c. é a tabela de ACESSO DE SAÍDA do REDEX no API RECINTOS
# A tabela e. é a tabela de ACESSO DE ENTRADA no TERMINAL EXPORTADOR (Operador Portuário)
# A lógica é que o caminhão com conteiner SAI DO REDEX ANTES DE ENTRAR NO TERMINAL EXPORTADOR (por isso o maior igual)
# Atenção! A DPW/EMBRAPORT pode operar como REDEX e envia carga para a BTP como pode receber carga de um REDEX e operar como Terminal exportador
# Este código contempla estes dois casos da DPW/EMBRAPORT

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,vazioConteiner
0,13005205,CUD4902,CUD3680,KOCU4334002,8931318,2025-10-09 10:11:00,C,E,12014047000193,10430518803,ALDO PASCOAL SOARES JUNIOR,,,0
1,13005211,MOY2542,BXH0752,GCNU1360149,8931318,2025-10-09 10:16:40,C,E,01779939000169,39655152847,DANILO VICENTE,35251003662398000110550010000309121714882500,,0
2,13005222,DVS9C58,TKL2H10,HLXU8781679,8931318,2025-10-09 10:27:25,C,E,21425093000176,44539833871,VINICIUS FRANCA COSTA DE LIRA,,152505300635701,0
3,13005227,GFM8B51,EJZ3C13,ACLU9721100,8931318,2025-10-09 10:34:18,C,E,10580796000152,30699765811,AMADEU HERCULANO DOS SANTOS,35251002261180000190550010000698821620708818,,0
4,13005231,LUD1D58,DBM4644,TEMU5777030,8931318,2025-10-09 10:38:40,C,E,12014047000193,13379861863,CICERO GOMES DOS SANTOS,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21677,13144653,3717937,PDR6468,SEGU6946437,8931404,2025-10-15 23:51:53,C,E,01417222000258,32178367828,JAMES MARCELO DOS SANTOS,"35251016404287004738550010016545591679393499, ...",,0
21678,13144668,EPT8H91,FTB3B22,MRSU8389512,8931404,2025-10-15 23:55:26,C,E,09621639000169,30288616820,MARCIO FERRARI,31251016849231001003550010000409831595705958,,0
21679,13144670,BWH2B64,CUA7J24,UACU3817310,8931404,2025-10-15 23:55:35,C,E,54834007000138,21760216810,ANDERSON SOARES DE OLIVEIRA,35250759106666000171550070000016611773041911,,0
21680,13144669,FEI3J28,DVS1D33,HAMU2291342,8931404,2025-10-15 23:55:42,C,E,13982346000184,69560870572,JOEL CORREIA DA CRUZ,35251000190373000920550020000061011056937615,,0


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,vazioConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,13004661,HOM6695,EFU5433,MSNU1458997,0,8931309,04489579810,JOSE ALMEIDA DE MENEZES,2025-10-09 11:36:36,S
1,13004669,NCP8J86,TLT3F49,MSNU3083138,0,8931309,28938218805,RODRIGO SEABRA DA SILVA,2025-10-09 11:53:59,S
2,13004674,MSJ1347,FTH2030,MSNU2345439,0,8931309,81569912491,HELIO SANTOS DE MELO,2025-10-09 12:07:47,S
3,13004672,TKW9D84,DAO7248,FCIU5976227,0,8931309,34283939889,JUSSIELIO DA SILVA CALISTRO,2025-10-09 12:09:29,S
4,13004678,JBS7C52,NXW2730,TGBU3203584,0,8931309,25210365875,EDIVALDO DE OLIVEIRA SILVA,2025-10-09 12:12:26,S
...,...,...,...,...,...,...,...,...,...,...
5836,13151623,FGA4A35,CUA6F32,HLBU1768209,0,8933204,30646933876,MICHAEL BRITO DOS SANTOS,2025-10-15 21:29:50,S
5837,13151638,CNR1J79,BSF4F24,FANU3809490,0,8933204,35177073840,SIDNEY DO NASCIMENTO CHAVES,2025-10-15 21:51:26,S
5838,13151639,IAF7G98,BSF4488,FANU1155678,0,8933204,37210381856,EDINALDO IZIDIO DE JESUS SOUZA,2025-10-15 21:56:11,S
5839,13151640,ESU6C27,JTL3C87,ONEU0729637,0,8933204,02394629550,DANILO BRANDAO FERREIRA,2025-10-15 21:57:44,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/5788 [00:00<?, ?it/s]

Registros: 5788, imagens: 2389


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,id_S_REDEX,placa_S_REDEX,vazioConteiner_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,13005291,DPE4H18,CUD2614,KOCU4795447,8931318,2025-10-09 11:45:23,C,E,12014047000193,29058448843,...,13017748,DPE4H18,0,8931404,29058448843,HUMBERTO PINHEIRO DA SILVA,2025-10-09 10:09:00,S,1.606389,68e8f563a2f66f3847eb43d0
1,13005300,EJW2C71,FRY4635,NYKU3696674,8931318,2025-10-09 11:50:04,C,E,12014047000193,26909352874,...,13017771,EJW2C71,0,8931404,26909352874,RUBENS LEAL DA SILVA,2025-10-09 10:21:07,S,1.482500,68e8f563a2f66f3847eb449e
2,13005312,CUA5065,CUA5F97,GCNU1281815,8931318,2025-10-09 12:06:15,C,E,02077058000169,01778290876,...,13076317,CUA5065,0,8932797,01778290876,JOSE NIVALDO DALAQUA,2025-10-09 11:12:27,S,0.896667,68e8f56bf37591d395e16bea
3,13005319,CUA4J91,CUA5944,GAOU2408991,8931318,2025-10-09 12:17:16,C,E,02077058000169,27769375800,...,13076324,CUA4J91,0,8932797,27769375800,MARCIO JOSE CARDOSO DOS SANTOS,2025-10-09 11:19:05,S,0.969722,68e8f56af37591d395e16bbe
4,13005322,CUD3J75,CUD3881,MSKU7147262,8931318,2025-10-09 12:24:02,C,E,12014047000193,27885045862,...,13017874,CUD3J75,0,8931404,27885045862,REGINALDO MENDONÃ?A DA SILVA,2025-10-09 10:54:14,S,1.496667,68e8f563a2f66f3847eb4456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,13144222,ESU6C27,JTL3C87,ONEU0729637,8931404,2025-10-15 22:45:40,C,E,58180316000192,02394629550,...,13151640,ESU6C27,0,8933204,02394629550,DANILO BRANDAO FERREIRA,2025-10-15 21:57:44,S,0.798889,None
5784,13144227,TLX7H88,FPQ6F91,MNBU4260226,8931404,2025-10-15 22:48:40,C,E,02916265009116,31859045839,...,13069117,FWH1F79,1,8932761,04971805885,GILSON CARLOS DE PAULA,2025-10-11 05:19:50,S,113.480556,None
5785,13144244,IAF7G98,BSF4488,FANU1155678,8931404,2025-10-15 22:54:02,C,E,58180316000192,37210381856,...,13151639,IAF7G98,0,8933204,37210381856,EDINALDO IZIDIO DE JESUS SOUZA,2025-10-15 21:56:11,S,0.964167,None
5786,13144246,EMU3430,DTA7G63,UETU7342286,8931404,2025-10-15 22:54:42,C,E,61139432001900,04205282885,...,13145961,EMU3430,0,8932754,04205282885,EXPEDITO JOSE DA SILVA,2025-10-15 22:09:39,S,0.750833,None


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
909,TJF0C94,HAMU1736201,8932796,2025-10-09 12:08:50,8931356,2025-10-10 15:25:36,27.279444,None,
923,FJU6J82,MRSU7394849,8931404,2025-10-09 11:50:08,8931356,2025-10-10 22:10:06,34.332778,None,
932,GII5E86,TEMU6589803,8931404,2025-10-09 21:37:27,8931356,2025-10-11 01:50:33,28.218333,None,
934,KOU1J22,ECMU4915559,8931404,2025-10-09 22:10:43,8931356,2025-10-11 03:10:52,29.002500,None,
1066,FZQ2G08,TRHU4868119,8931404,2025-10-09 23:51:39,8931356,2025-10-13 04:42:08,76.841389,None,
1068,RAF3F03,MRSU7087478,8931404,2025-10-09 11:55:07,8931356,2025-10-13 05:50:01,89.915000,None,
1234,BHN9361,CMAU4393359,8931404,2025-10-11 01:04:31,8931356,2025-10-13 14:09:23,61.081111,None,
1310,BYP8A46,MRSU5688588,8931404,2025-10-09 18:22:03,8931356,2025-10-13 16:44:30,94.374167,None,
1359,BXD9G36,CAAU5228760,8931404,2025-10-10 16:32:07,8931356,2025-10-13 18:23:00,73.848056,None,
1373,DJG2H93,MSKU0878521,8931404,2025-10-10 20:03:21,8931356,2025-10-13 19:21:59,71.310556,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"